In [1]:
using CSV
using DataFrames
using Statistics

In [2]:
results = CSV.read("simple_tests.csv", DataFrame, header = [
    "hostname",
    "job_id",
    "commit_id",
    "xrt_version",
    "bdf",
    "rank",
    "repetition",
    "frame_size",
    "message_size",
    "iterations",
    "test_nfc",        
    "transmission_time",
    "failed_transmissions",
    "status_not_ok_count",
    "fifo_rx_overflow_count",
    "fifo_tx_overflow_count",
    "byte_errors",
    "frames_received",
    "frames_with_errors"
])

results.fpga = results.hostname .* "_" .* results.bdf 
results.port = results.fpga .* "_" .* string.(results.rank .% 2)
results.latency = results.transmission_time ./ results.iterations
results.throughput = results.message_size ./ results.latency
results.throughput_gbit_s = results.throughput * 8 / 1e9

results

In [ ]:
function check(df, by)
    sort(
        combine(
            groupby(df, by),
            nrow => :count,
            :failed_transmissions => sum => :failed_transmissions,
            :byte_errors => sum => :byte_errors,
            :frames_with_errors => sum => :frame_errors,
            :status_not_ok_count => sum => :status_errors,
            :fifo_rx_overflow_count => sum => :fifo_rx_overflows,
            :fifo_tx_overflow_count => sum => :fifo_tx_overflows,
        ),
        by
    )
end

print(check(results, :hostname))
print(check(results, :fpga))
print(check(results, :port))


In [ ]:
function check_error_type(df, by)
    sort(
        unstack(
            combine(
                groupby(df, [by, :failed_transmissions]),
                nrow => :count
            ),
            :failed_transmissions,
            :count,
        ),
        by
    )
end

print(check_error_type(results, :hostname))
print(check_error_type(results, :fpga))
print(check_error_type(results, :port))


In [ ]:
function aggregate_means(df)
    combine(
        groupby(df, [:frame_size, :message_size]),
        :latency => mean => :latency,
        :throughput => mean => :throughput,
        :throughput_gbit_s => mean => :throughput_gbit_s,
        nrow => :count,
        :failed_transmissions => sum => :failed_transmissions,
        :byte_errors => sum => :byte_errors,
        :frames_with_errors => sum => :frame_errors,
    )
end

means = aggregate_means(results)

function tabulate(df, value)
    unstacked = unstack(
        df,
        :message_size,
        :frame_size,
        value
    )
    unstacked[:,vcat(
        ["message_size"],
        sort(setdiff(names(unstacked), ["message_size"]), by = fs -> parse(Int, fs))
    )]

end



println("Results")
println(tabulate(means, :count))
println("Failed transmissions")
println(tabulate(means, :failed_transmissions))
println("Byte errors")
println(tabulate(means, :byte_errors))
println("Frame errors")
println(tabulate(means, :frame_errors))
println("Latency")
println(tabulate(means, :latency))
println("Throughput")
println(tabulate(means, :throughput_gbit_s))